# Импорты

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import json

# Чтение исходных данных

In [27]:
DATAPATH = './data/'

In [28]:
train_file = DATAPATH + 'train.json'
test_file = DATAPATH + 'test.json'

In [29]:
sample = pd.read_csv(DATAPATH + 'sample.csv')
sample.head()

,id,sentiment
0,0,neutral
1,1,positive
2,2,negative
3,3,positive
4,4,neutral


In [30]:
def get_df(json_filename):
    with open(json_filename) as json_file:
        temp = json.load(json_file)
        return pd.DataFrame.from_records(temp)

In [31]:
train = get_df(train_file)
train.head()

,text,id,sentiment
0,Досудебное расследование по факту покупки ЕНПФ...,1945,negative
1,Медики рассказали о состоянии пострадавшего му...,1957,negative
2,"Прошел почти год, как железнодорожным оператор...",1969,negative
3,По итогам 12 месяцев 2016 года на территории р...,1973,negative
4,Астана. 21 ноября. Kazakhstan Today - Агентств...,1975,negative


In [32]:
test = get_df(test_file)
test.head()

,text,id
0,"Как сообщает пресс-служба акимата Алматы, для ...",0
1,Казахстанские авиакомпании перевозят 250 тысяч...,1
2,На состоявшемся под председательством Касым-Жо...,2
3,В ОАЭ состоялись переговоры между казахстанско...,3
4,12 вагонов грузового поезда сошли с путей в Во...,4


# Предобработка с помощью UDPipe

## Установка нужных пакетов и загрузка модели

In [33]:
!pip install ufal.udpipe

     |████████████████████████████████| 307kB 1.3MB/s eta 0:00:01
  Created wheel for ufal.udpipe: filename=ufal.udpipe-1.2.0.3-cp37-cp37m-linux_x86_64.whl size=5748613 sha256=470353ccec5b3001b81bb3340387d302b4f3fbb6dfe72fa019fa127ea2c56f44
  Stored in directory: /home/rprilepskiy/.cache/pip/wheels/0c/9d/db/6d3404c33da5b7adb6c6972853efb6a27649d3ba15f7e9bebb
Successfully built ufal.udpipe


In [35]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-cp37-none-any.whl size=9680 sha256=b528b2b12658ad8df192a555b3eb12203fec29bc9d24f974ce1e618a818a1bf0
  Stored in directory: /home/rprilepskiy/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [39]:
import wget

udpipe_url = 'https://rusvectores.org/static/models/udpipe_syntagrus.model'
text_url = 'https://rusvectores.org/static/henry_sobolya.txt'

modelfile = wget.download(udpipe_url)
textfile = wget.download(text_url)